#### Tensorflow 6강. File load in Tensorflow and Queue Thread
##### https://www.youtube.com/watch?v=hPkmxczEj6k&index=6&list=PL1H8jIvbSo1qOtjQXFzBxMWjL_Gc5x3yG&t=940s

In [1]:
import tensorflow as tf
import os
from PIL import Image

In [2]:
image_dir = os.getcwd()+"/Test_Dataset_png"
image_list = os.listdir(image_dir)
image_list.sort()
#print(image_list[:5])
#print(len(image_list))

for i in range(len(image_list)):
    image_list[i] = image_dir + image_list[i];
#print(image_list[:5])


label_dir=os.getcwd()+"/Test_Dataset_csv/label.csv"
label_list = [label_dir]

imagenmae_queue = tf.train.string_input_producer(image_list)
labelnmae_queue = tf.train.string_input_producer(label_list)

image_reader = tf.WholeFileReader()
label_reader = tf.TextLineReader()

image_key, image_value = image_reader.read(imagenmae_queue)
label_key, label_value = label_reader.read(labelnmae_queue)

#image =tf.image.decode_png(image_value)
#label =tf.decode_csv(label_value,record_defaults= [[0]])

image_decoded =tf.cast(tf.image.decode_png(image_value), tf.float32)
label_decoded =tf.cast(tf.decode_csv(label_value,record_defaults= [[0]]),tf.float32)

label = tf.reshape(label_decoded,[1])
image = tf.reshape(image_decoded,[61,49,1])

x, y_ = tf.train.shuffle_batch(tensors=[image,label], batch_size= 32, 
                              num_threads=4, capacity= 50000, min_after_dequeue=10000)                                  

#for i in range(len(y_)):                              

#x= tf.cast(image, tf.float32)
#y_ = tf.cast(label, tf.float32)
#y_ = tf.reshape(y_,[-1,1])

In [3]:
print(x)
print(y_)

Tensor("shuffle_batch:0", shape=(32, 61, 49, 1), dtype=float32)
Tensor("shuffle_batch:1", shape=(32, 1), dtype=float32)


In [4]:
print(y_)

Tensor("shuffle_batch:1", shape=(32, 1), dtype=float32)


In [5]:
image_width = 49
image_hight = 61

In [6]:
#x = tf.placeholder(tf.float32,shape=[None,image_width,image_hight ])
#y_ = tf.placeholder(tf.float32,shape=[None,1]) # 정답데이터가 몇 차원

In [7]:
# hidden1 convolution
hidden1_w = tf.Variable(tf.truncated_normal([5,5,1,32]))
hidden1_b = tf.Variable(tf.zeros([32]))

x_image = tf.reshape(x, [-1, image_width, image_hight,1])

h_conv1=tf.nn.relu(tf.nn.conv2d(x_image, hidden1_w, strides=[1,1,1,1], padding="SAME" ) +hidden1_b)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1],strides=[1, 1, 1, 1], padding='SAME')

In [8]:
# hidden2 convolution
hidden2_w = tf.Variable(tf.truncated_normal([5,5,32,64]))
hidden2_b = tf.Variable(tf.zeros([64]))

h_conv2=tf.nn.relu(tf.nn.conv2d(h_pool1, hidden2_w, strides=[1,1,1,1], padding="SAME" ) + hidden2_b)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1],strides=[1, 1, 1, 1], padding='SAME')

In [9]:
h_flat = tf.reshape(h_pool2, [-1, 49*61*64])
fc_w = tf.Variable(tf.truncated_normal([49*61*64,10]))
fc_b = tf.Variable(tf.zeros([10]))

h_fc1= tf.nn.relu(tf.matmul(h_flat, fc_w) + fc_b)
keep_prob = tf.placeholder(tf.float32)
drop_fc = tf.nn.dropout(h_fc1, 0.7)

In [10]:
print(h_flat)
print(fc_w)
print(fc_b)
print(h_fc1)

Tensor("Reshape_3:0", shape=(32, 191296), dtype=float32)
Tensor("Variable_4/read:0", shape=(191296, 10), dtype=float32)
Tensor("Variable_5/read:0", shape=(10,), dtype=float32)
Tensor("Relu_2:0", shape=(32, 10), dtype=float32)


In [11]:
out_w = tf.Variable(tf.truncated_normal([10,1]))
out_b = tf.Variable(tf.zeros([1]))

#pred = tf.matmul(h_fc1, out_w) + out_b
pred = tf.matmul(drop_fc, out_w) + out_b

In [12]:
#cross_entropy 
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(pred, y_))
train = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [13]:
#[T,T,F,F,F,F,T] BOOL
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)    
    sess.run(tf.global_variables_initializer())
    

    for i in range(100):
        #sess.run(train)   
        #cost, acc =  sess.run([loss, accuracy])       
        #if 학습할때
        #cost, acc =  sess.run([loss, accuracy], {keep_prob:0.7})
        #IF flfflwm ahem
        #result = ssess.run(pred, {keep_prob:1.0})
        sess.run(train)   
        cost, acc =  sess.run([loss, accuracy])     
        print("cost:",cost)
        print("acc:", acc) 

            
  
    coord.request_stop()
    coord.join(threads)

OutOfRangeError: RandomShuffleQueue '_2_shuffle_batch/random_shuffle_queue' is closed and has insufficient elements (requested 32, current size 0)
	 [[Node: shuffle_batch = QueueDequeueMany[_class=["loc:@shuffle_batch/random_shuffle_queue"], component_types=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch/random_shuffle_queue, shuffle_batch/n)]]

Caused by op 'shuffle_batch', defined at:
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Program Files\Anaconda3\lib\site-packages\traitlets\config\application.py", line 653, in launch_instance
    app.start()
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in